# Introduction to WaterTAP Costing

# DELETE THIS CELL BEFORE MERGING
Outline:
- Brief summary of WaterTAP costing - slides will hopefully cover most of the details
- Just build an RO flowsheet
- Walk through adding the existing costing for each unit model (consider showing snippets of the unit model costing files or documentation with our explanations)
- Walk through how to modify the existing parameters
- How to add system level metrics (LCOW, SEC, etc.)
- Walk through how to go from simulating costing to optimizing it (add objective function for minimizing LCOW)
- Add a try it yourself for optimizing for a different objective (SEC)
- Walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html

## TODO: Mutka should look into adding capital cost components dynamically


This tutorial will demonstrate how to add costing to a simple RO flowsheet as well as how to compute and optimize system level metrics like levelized cost of water (LCOW), specific energy consumption (SEC), and annual water production.

## Step 1: Import Modules

In [1]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)
from pyomo.network import Arc

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.initialization import propagate_state
from idaes.core import UnitModelCostingBlock
from idaes.models.unit_models import Feed, Product

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.costing import WaterTAPCosting
from watertap.costing.unit_models.reverse_osmosis import cost_reverse_osmosis
from watertap.costing.unit_models.pump import cost_high_pressure_pump
from watertap.core.solvers import get_solver

## Step 2: Build Flowsheet

In the following cell, we will recreate the flowsheet made in the Week 2 tutorial titled `RO_flowsheet_build`.

<p align="center">
<img src='../week2/graphics/RO_flowsheet.png' alt="RO PFD" width="1000" height=800>
</p>

In [2]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build feed unit model
m.fs.feed = Feed(property_package=m.fs.properties)

# Build RO unit model 
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

# Build pump unit model
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Build product and brine streams
m.fs.product = Product(property_package=m.fs.properties)
m.fs.brine = Product(property_package=m.fs.properties)

# Create Arcs
m.fs.feed_to_pump = Arc(source=m.fs.feed.outlet, destination=m.fs.pump.inlet)
m.fs.pump_to_RO = Arc(source=m.fs.pump.outlet, destination=m.fs.RO.inlet)
m.fs.RO_to_product = Arc(source=m.fs.RO.permeate, destination=m.fs.product.inlet)
m.fs.RO_to_brine = Arc(source=m.fs.RO.retentate, destination=m.fs.brine.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# Feed, 4 degrees of freedom
m.fs.feed.properties[0].flow_vol_phase["Liq"].fix(1e-3)
m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(35)
m.fs.feed.properties[0].pressure.fix(101325)
m.fs.feed.properties[0].temperature.fix(273.15 + 25)

# Pump, 2 degrees of freedom
m.fs.pump.efficiency_pump.fix(0.80)
m.fs.pump.control_volume.properties_out[0].pressure.fix(75 * pyunits.bar)

# RO unit, 7 degrees of freedom
m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(3.5e-8)
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)
m.fs.RO.feed_side.velocity[0, 0].fix(0.15)
m.fs.RO.feed_side.channel_height.fix(1e-3)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)
m.fs.RO.permeate.pressure[0].fix(101325)

# Set default property values
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))

# Set unit model values
set_scaling_factor(m.fs.pump.control_volume.work, 1e-3)
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate and propagate scaling factors
calculate_scaling_factors(m)

# Get WaterTAP solver
solver = get_solver()

# solve feed
solver.solve(m.fs.feed)

# Propagate state from feed to pump
propagate_state(m.fs.feed_to_pump)
# Initialize pump
m.fs.pump.initialize()

# Propagate state from pump to RO
propagate_state(m.fs.pump_to_RO)
# Initialize RO
m.fs.RO.initialize()

# Propagate state from RO to brine and product
propagate_state(m.fs.RO_to_brine)
propagate_state(m.fs.RO_to_product)

# Initialize brine and product
m.fs.brine.initialize()
m.fs.product.initialize()

2025-11-29 20:09:37 [INFO] idaes.init.fs.pump.control_volume.properties_out: fs.pump.control_volume.properties_out State Released.
2025-11-29 20:09:37 [INFO] idaes.init.fs.pump.control_volume: Initialization Complete
component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
2025-11-29 20:09:37 [INFO] idaes.init.fs.pump.control_volume.properties_in: fs.pump.control_volume.properties_in State Released.
2025-11-29 20:09:37 [INFO] idaes.init.fs.pump: Initialization Complete: optimal - Optimal Solution Found
2025-11-29 20:09:37 [INFO] idaes.init.fs.RO.feed_side.properties_out: fs.RO.feed_side.properties_out State Released.
2025-11-29 20:09:37 [INFO] idaes.init.fs.RO.feed_side.properties_interface: fs.RO.feed_side.properties_interface State Released.
2025-11-29 20:09:37 [INFO] idaes.init.fs.RO.feed_side: Initialization Complete
2025-11-29 20:09:37 [INFO] idaes.init.fs.RO.permeate_side: fs.RO.permeate_side State R

## Step 3: Add RO Costing

### Step 3a: Create the Costing Model

`WaterTAPCosting` contain extensions, methods, and variables and constraints common to all WaterTAP Costing Packages. In other words, this sets up the base cost model, which we will expand upon by adding RO-specific cost terms.

In [3]:
# Set up WaterTAP Costing
m.fs.costing = WaterTAPCosting()

# Set units of base currency
m.fs.costing.base_currency = pyunits.USD_2020

### Step 3b: Add RO Unit Model Costing Block
Next, add the `UnitModelCostingBlock` for RO, which will set up all the RO-specific variables and constraints related to costing the unit. There are two configuration options on `UnitModelCostingBlock` that must be set. The first is `flowsheet_costing_block`, which refers to the instance of WaterTAP costing. This is conventionally referred to as `m.fs.costing`, and most flowsheets will only have one instance of WaterTAP costing; however, complicated flowsheets may contain multiple. The second configuration option is `costing_method`, which allows the user to choose between different cost assumptions. As shown  below, RO has two costing methods:

<p align="center">
<img src='graphics/ro_costing_doc.png' alt="RO Costing Assumptions" width="1000" height=800>
</p>

In [4]:
m.fs.RO.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=cost_reverse_osmosis,
    costing_method_arguments={"ro_type": "high_pressure"},
)

# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True : False :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    30 :  None :  True : False :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    75 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Step 4: Modify Existing Parameters (if necessary)

Let's say we want to change the membrane unit cost in the conventional case from 75 USD/m2 to 60 USD/m2. The naming convention for updating these parameters is `m.fs.costing.UnitModel_CostingMethod.Parameter.fix(new_value)`

In [5]:
# Update RO membrane cost
m.fs.costing.reverse_osmosis.high_pressure_membrane_cost.fix(60)

# Initialize costing
# m.fs.costing.initialize()

# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True :  True :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    30 :  None :  True :  True :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    60 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Step 5: Solve the model

In [6]:
# Solve model
results = solver.solve(m)
assert_optimal_termination(results)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

Solver Status: ok, Termination Condition: optimal


## Try It Yourself

Add a `UnitModelCostingBlock` for a high-pressure pump and change the cost from 1.908 USD/W to 1 USD/W given the following information. Afterwards, re-solve the model.
<p align="center">
<img src='graphics/pump_costing_doc.png' alt="Pump Costing Assumptions" width="1000" height=800>
</p>

In [7]:
# Add pump UnitModelCostingBlock
m.fs.pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=cost_high_pressure_pump,
)

# Print the cost model for the pump before updating
m.fs.costing.high_pressure_pump.display()

# Update pump cost parameter
m.fs.costing.high_pressure_pump.cost.fix(1)

results = solver.solve(m)
assert_optimal_termination(results)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

# Print the cost model for the pump to verify changes
m.fs.costing.high_pressure_pump.display()

Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 1.908 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None
Solver Status: ok, Termination Condition: optimal
Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     1 :  None :  True :  True :  Reals

  Objectives:
    None

  Constraints:
    None


## Step 6: Add System-Level Metrics

Now we want to add system-level metrics to the cost model, such as levelized cost of water (LCOW), specific energy consumption (SEC), and annual water production.

In [8]:
m.fs.costing.cost_process()

# Add levelized cost of water based on the feed stream flow
m.fs.costing.add_LCOW(m.fs.feed.properties[0].flow_vol)

# Add annual water production
m.fs.costing.add_annual_water_production(m.fs.product.properties[0].flow_vol)

# Re-solve system
results = solver.solve(m)

# Print LCOW value
print(f"Levelized cost of water: {value(m.fs.costing.LCOW)} $/m3")

Levelized cost of water: 0.3136567840551696 $/m3


## Step 7: Optimize LCOW
Set an objective to minimize the levelized cost of water and re-solve the system.

In [9]:
m.fs.objective = Objective(expr=m.fs.costing.LCOW)

results = solver.solve(m)
assert_optimal_termination(results)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

# Display costing results
m.fs.costing.display()

# Print LCOW value
print(f"Levelized cost of water: {value(m.fs.costing.LCOW)} $/m3")

Solver Status: ok, Termination Condition: optimal
Block fs.costing

  Variables:
    total_investment_factor : Total investment factor [investment cost/equipment cost]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   1.0 :  None :  True :  True :  Reals
    maintenance_labor_chemical_factor : Maintenance-labor-chemical factor [fraction of equipment cost/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.03 :  None :  True :  True :  Reals
    utilization_factor : Plant capacity utilization [fraction of uptime]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.9 :  None :  True :  True :  Reals
    electricity_cost : Electricity cost
        Size=1, Index=None, Units=USD_2018/kWh
        Key  : Lower : Value : Upper : Fixed : Stale : Dom

## Try It Yourself

Use `add_specific_energy_consumption` to add SEC to the system-level metrics, set an objective to minimize SEC, re-solve the system, and print the costing results.

In [10]:
# Add specifc energy consumption based on the feed stream flow
m.fs.costing.add_specific_energy_consumption(m.fs.feed.properties[0].flow_vol)

# Set the objective for minimizing specific energy consumption
m.fs.objective = Objective(expr=m.fs.costing.specific_energy_consumption)

results = solver.solve(m)
assert_optimal_termination(results)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

# Print SEC value
print(f"Specific energy consumption: {value(m.fs.costing.specific_energy_consumption)} kWh/m3")

'pyomo.core.base.objective.ScalarObjective'>) on block fs with a new Component
(type=<class 'pyomo.core.base.objective.AbstractScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
Solver Status: ok, Termination Condition: optimal
Specific energy consumption: 2.5689843749999994 $/kW


## Step 7: Register Custom Flows

Could walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html, but after some further reflection, I think this is best left for a post-session tutorial. This tutorial is already pretty content-heavy as it currently stands